In [1]:
running = False
try:
    sock.close()
except:
    pass

In [2]:
from parser import public_construct, public_match

In [3]:
from utils import SocketReader

In [4]:
import math

In [5]:
rmeote_offer = """v=0
o=- 5011011625244685403 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 59190 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 35.222.40.173
a=candidate:2334919121 1 udp 2113937151 dde5f7da-d6c8-4154-a586-7a8886fb9c76.local 59190 typ host generation 0 network-cost 999
a=candidate:2670179031 1 udp 1677729535 35.222.40.173 59190 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:8sC0
a=ice-pwd:OwnMi2c7HonDPORmpeR96W3g
a=ice-options:trickle
a=fingerprint:sha-256 E0:D8:A2:3B:54:91:BF:55:42:CC:C0:D5:73:7E:98:CA:4E:83:64:3A:EF:42:6C:5A:2F:A8:66:4E:39:52:20:18
a=setup:actpass
a=mid:0
a=sctp-port:5000
a=max-message-size:262144

"""

In [6]:
for line in rmeote_offer.splitlines():
    if 'a=candidate:' in line and '.local' not in line:
        res = line.split(' ')
        target_ip = res[4]
        target_port = int(res[5])
    if 'a=ice-ufrag:' in line:
        remote_ufrag = line.split(':')[1]
    if 'a=ice-pwd:' in line:
        remote_pwd = line.split(':')[1]

target_ip_num = 0
for i, seg in enumerate(target_ip.split('.')):
    target_ip_num += int(seg) * (256**(3-i))

target_ip, target_port, remote_ufrag, remote_pwd

('35.222.40.173', 59190, '8sC0', 'OwnMi2c7HonDPORmpeR96W3g')

In [7]:
import random, string
len_remote_ufrag = len(remote_ufrag)
# len_remote_ufrag = 4 # chrome
local_ufrag = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16 - len_remote_ufrag - 1))
local_pwd = 'AgLw44VDUK531eYVCMpZMAjF'
local_ufrag

'P43tcNjYJ3V'

In [8]:
from stun import do_stun
import random
sport = random.randrange(30000, 50000)
ip, port = do_stun(sport)
f"found {ip}:{port} while source port is {sport}"

'found 143.178.72.250:44711 while source port is 44711'

In [10]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
import OpenSSL.crypto

own_cert_private_key = ec.generate_private_key(ec.SECP256R1())

own_cert_private_key_bytes = own_cert_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)


privkey = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, own_cert_private_key_bytes)

pubkey = OpenSSL.crypto.load_publickey(OpenSSL.crypto.FILETYPE_PEM,
                                     own_cert_private_key.public_key().public_bytes(
                                         encoding=serialization.Encoding.PEM,
                                         format=serialization.PublicFormat.SubjectPublicKeyInfo,
                                     )
                                    )

local_cert = OpenSSL.crypto.X509()
local_cert.set_version(2)
local_cert.gmtime_adj_notBefore(0)
local_cert.gmtime_adj_notAfter(1 * 24 * 60 * 60)
local_cert.get_issuer().CN = 'WebRTC'
local_cert.get_subject().CN = 'WebRTC'
local_cert.set_serial_number(random.randrange(1<<63, 1<<64))
local_cert.set_pubkey(pubkey)
local_cert.sign(privkey, 'sha256') 
print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, local_cert).decode('utf-8')) 
local_cert_sha = local_cert.digest('sha256').decode('utf-8')
local_cert_sha

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            fc:64:0c:5a:84:1a:99:53
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jun 26 12:27:26 2024 GMT
            Not After : Jun 27 12:27:26 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:d5:74:22:47:46:37:41:5e:31:8a:a1:73:de:94:
                    23:a6:c6:f9:02:07:c3:39:6e:01:25:c8:b8:d3:44:
                    e3:35:b6:01:4c:d6:4c:39:1b:45:60:58:e6:9f:16:
                    15:75:23:9e:e9:de:1f:40:5d:81:09:60:10:48:8d:
                    72:e7:9d:a5:7e
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:45:02:21:00:f4:6d:57:e4:79:cf:c4:a5:00:c9:a8:24:b9:
        1d:60:78:76:3c:f5:0e:5d:8e:db:c6

'11:12:AF:87:F1:49:63:76:73:84:11:3E:23:02:F1:57:3F:ED:DA:D1:66:BF:71:8E:F4:F4:6A:23:2C:DC:3B:71'

In [11]:
local_offer = f"""v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application {port} UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 {ip}
a=candidate:3055715734 1 udp 1677729535 {ip} {port} typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:{local_ufrag}
a=ice-pwd:{local_pwd}
a=ice-options:trickle
a=fingerprint:sha-256 {local_cert_sha}
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144
"""
print(local_offer)

v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 44711 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 143.178.72.250
a=candidate:3055715734 1 udp 1677729535 143.178.72.250 44711 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:P43tcNjYJ3V
a=ice-pwd:AgLw44VDUK531eYVCMpZMAjF
a=ice-options:trickle
a=fingerprint:sha-256 11:12:AF:87:F1:49:63:76:73:84:11:3E:23:02:F1:57:3F:ED:DA:D1:66:BF:71:8E:F4:F4:6A:23:2C:DC:3B:71
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144



In [12]:
stun_header_spec = """
2@zeroes
-14@message_type
-16@message_length
-32@magic_cookie
-96@transaction_id
"""

padding_0_attribute_spec = """
...(
    16==0x20?16@xor_mapped_ip(
        8@zeroes
        -8@family
        -16@port
        -32@ip
    )
    :16==0x06?16@username()
    :16==0x08?16@message_integrity(20B@hmac)
    :16==0x25?16@use_candidate()
    :16==0x8029?16@ice_controlled(8B@tie_breaker)
    :16==0x802a?16@ice_controlling(8B@tie_breaker)
    :16==0x24?16@priority(4B@priority)
    :16==0x8028?16@fingerprint(4B@crc32)
    :16@selector-16@data()
)
"""

import random
magic_cookie = 0x2112A442 # constant

In [13]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hmac import HMAC

def get_message_integrity(msg, key):
    msg = bytearray(msg)
    msg[3] = len(msg) + 24 - 20
    hm = HMAC(key, hashes.SHA1())
    hm.update(msg)
    return hm.finalize()

def append_message_integrity(msg, key):
    hmac = get_message_integrity(msg, key)
    attrs = [{"message_integrity": {"hmac": hmac}, "_selector": 0x08}]
    msg = bytearray(msg)
    msg += public_construct(padding_0_attribute_spec, attrs)
    msg[3] = len(msg) - 20
    return msg

import binascii
def append_fingerprint(msg):
    msg = bytearray(msg)
    msg[3] = len(msg) + 8 - 20
    crc32 = binascii.crc32(msg) ^ 0x5354554e
    attrs = [{"fingerprint": {"crc32": crc32}, "_selector": 0x8028}]
    msg += public_construct(padding_0_attribute_spec, attrs)
    return msg
    


# v = """000100502112a4427139396c466f2b2f642f48650006000952556d333a4e6b4b36000000c0570004000003e7802a00084f02cc537c6985ff00250000002400046e001eff00080014ed4243520504f6eeefe90146a25142054ab03a4380280004f0c40a27"""
# msgm = bytes.fromhex(v)
# should_be = msgm[-28:-8].hex()
# print('should be', should_be)
# msgm = bytearray(msgm[:-8][:-24])
# key = b'7bjW8vqNNZPvdFDCa3IJ3J7E'
# print(v)
# print(append_fingerprint(append_message_integrity(msgm, key)).hex())


In [14]:
def create_response(transaction_id):
    attributes = [
        {'xor_mapped_ip': {'zeroes': 0x00,
       'family': 0x01,
       'port': target_port  ^ (magic_cookie >> 16),
       'ip': target_ip_num  ^ (magic_cookie)},
      '_selector': 0x0020}
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000100000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": transaction_id,
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, local_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg

def create_request():
    attributes = [
        {"username": remote_ufrag+':'+local_ufrag, "_selector": 0x06},
        # {"use_candidate": "", "_selector": 0x25},
        # {"ice_controlling": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x802a},
        {"ice_controlled": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x8029},
        {"priority": {"priority": 0x6e001eff}, "_selector": 0x24},
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000000000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": random.randrange(1<<96),
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, remote_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg


In [15]:
try:
    sock.close()
except:
    pass

sock = SocketReader(target_ip, target_port, 'udp')

sock.connect(source_port=sport)

In [16]:
running = True
import time
from queue import Queue

dts_queue = Queue(maxsize=10)

last_received = time.time()

def handler():
    while running:
        resp = sock.read_once()
        if resp[1] == 0xfe and resp[2] == 0xfd:
            dts_queue.put(resp)
        res, rem = public_match(stun_header_spec, resp)
        if res['message_type'].int() != 0b00000000000001:
            continue
        pkg = create_response(res['transaction_id'])
        sock.send(pkg)
        print('+', end='')
        last_received = time.time()

def sender():
    while running:
        sock.send(create_request())
        print('*', end='')
        time.sleep(2 if time.time() - last_received < 6 else 0.2)

import threading
threading.Thread(target=handler).start()
threading.Thread(target=sender).start()

*+++++*+*****+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*****

Exception in thread Thread-5 (handler):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6512/4249774954.py", line 11, in handler
  File "/home/mie/notebooks/utils.py", line 34, in read_once
    self._read_to_buffer()
  File "/home/mie/notebooks/utils.py", line 21, in _read_to_buffer
    buffer = self._read()
             ^^^^^^^^^^^^
  File "/home/mie/notebooks/utils.py", line 76, in _read
    return self.sock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^
ConnectionRefusedError: [Errno 111] Connection refused


*

Exception in thread Thread-6 (sender):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6512/4249774954.py", line 24, in sender
  File "/home/mie/notebooks/utils.py", line 17, in send
    self.sock.sendall(data)
ConnectionRefusedError: [Errno 111] Connection refused


In [17]:
# running = False
# sock.close()


In [18]:
transcript = b''

In [19]:
handshake_spec = """
8==1?24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -8@cookie()
    -16@suites(...16)
    -8@compression()
    -16@extensions(
        ...(
            16==23?16@extended_master_secret()
            :16==0xff01?16@renegotiation_info(8@renegotiation_info())
            :16==10?16@ext_supported_curves(
                16@data(...16)
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==13?16@ext_signature_algorithms(
                16@data(...16)
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==2?24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16@extensions(
        ...(
            16==23?16@ext_master_secret()
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==11?24@certificates_data(
    24@certificates(
        ...(
            24@certificate()
        )
    )
)
:8==12?24@server_key_exchange(
    8@curve_type
    -16@named_curve
    -8@pubkey()
    -16@sig_algorithm
    -16@signature()
)
:8==13?24@certificate_request(
    8@cert_types(...(8@cert_type))
    -16@sig_hash_algs(...(16@sig_hash_alg))
    -16@distinguished_names()
)
:8==14?24@server_hello_done()
:8==16?24@client_key_exchange(
    8@pubkey()
)
:8==15?24@certificate_verify(
    16@sig_algorithm
    -16@signature()
)
:8==4?24@new_session_ticket(
    4B@lifetime_hint
    -16@session_ticket()
)
:8==20?24@finished(12B@checksum)
"""

fragment_spec = """
4B@type_and_length
-16@message_sequence
-24@fragment_offset
-24@fragment()
"""

dtls_record_spec = """
8==22?16@tls_version-16@epoch-6B@sequence_number-16@tls_handshake(%s)
:8==20?16@tls_version-16@epoch-6B@sequence_number-16@change_cipher_spec_message()
"""%fragment_spec

In [20]:
client_hello = {
    'client_hello': {
        'version': 0xfefd,
        'random': bytes.fromhex('2dd93a4dd171b16cef3fd22029ab20c59938934d74b4c597b762f16ff261735b'),
        'session_id': '',
        'cookie': '',
        # 'suites': [0xc02b],
        'suites': [0xc02b, 0xc02f, 0xcca9, 0xcca8, 0xc009, 0xc013, 0xc00a, 0xc014, 0x009c, 0x002f, 0x0035],
        'compression': 0x00,
        'extensions': [
            {
                'extended_master_secret': '', '_selector': 23
            },
            {
                'renegotiation_info': {
                    'renegotiation_info': '',
                }, '_selector': 0xff01
            },
            {
                'ext_supported_curves': {
                    # 'data': [0x001d]
                    'data': [0x001d, 0x0017, 0x0018]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': 0x00
                }, '_selector': 0x000b
            },
            {
                'ext_session_tickets': '', '_selector': 35
            },
            {
                'ext_signature_algorithms': {
                    # 'data': [0x0403]
                    'data': [0x0403, 0x0804, 0x0401, 0x0503, 0x0805, 0x0501, 0x0806, 0x0601, 0x0201]
                }, '_selector': 0x000d
            },
            {
                'use_srtp': {
                    # 'data': [0x0001],
                    'data': [0x0001, 0x0008, 0x0007],
                    'mki': ''
                }, '_selector': 14
            }
        ]
    },
    '_selector': 0x01
}

client_hello_encoded = public_construct(handshake_spec, client_hello)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_hello_encoded[:4],
        'message_sequence': 0,
        'fragment_offset': 0,
        'fragment': client_hello_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 0,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)


In [21]:
transcript += dtls_record_encoded[13:]
sock.send(dtls_record_encoded)

In [22]:
# msg = bytes.fromhex("16fefd00000000000000000050020000440000000000000044fefd666d5aac48243def6a5466b05ba62cbf15f4eea136c99df59f782732ef9d558900c02b00001c00170000ff01000100000b0002010000230000000e0005000200010016fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda0030201020209008bff228a3f27c618300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303932315a170d3234303731353039303932315a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004a7990ef7c6e5f1a8c29e5cdb4b3e4cb1dc376b48ffd392751dc6fd763de583ee77a65da77341fbeed4e83fb1a09d4f1c621c89d9d974e50a0339ed4d380c46bb300a06082a8648ce3d0403020348003045022100a07ff3282ccb96a42387a8851d0b26a42a5f1082789fe8f682df50fbccb35a8202201dc64921ff9358f5bdec2a644c0597312f5cea62f295b875daf0e6bb1fe3c3e916fefd0000000000000002007c0c000070000200000000007003001d20eb7088c2a60f227ef9bdb01a5082a34048ff182b9e142afcb40c07b368fea016040300483046022100fec9df11479090a9553f1497c7707e4146100af1f9e5f36c60f9a4c421c5f419022100e494e6bb2765926893edd08f028cf692d69e7534affa89756802768a7bdc956816fefd000000000000000300250d00001900030000000000190201400012040308040401050308050501080606010201000016fefd0000000000000004000c0e0000000004000000000000")
msg = dts_queue.get()

In [23]:
packet, rem = public_match(dtls_record_spec, msg)
transcript += msg[13:-len(rem)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000000,
 'tls_handshake': {'type_and_length': 0x02000044,
  'message_sequence': 0x0000,
  'fragment_offset': 0x000000,
  'fragment': 0xfefd667c0941842af5001e938ed8545a934a23fd6877fbcc6eb9966a5ec6ea5bc11300c02b00001c00170000ff01000100000b0002010000230000000e00050002000100}}

In [24]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_hello, _ = public_match(handshake_spec, record)
server_hello

{'server_hello': {'version': 0xfefd,
  'random': 0x667c0941842af5001e938ed8545a934a23fd6877fbcc6eb9966a5ec6ea5bc113,
  'session_id': '',
  'suite': 0xc02b,
  'compression': 0x00,
  'extensions': [{'ext_master_secret': '', '_selector': 0x0017},
   {'ext_renegotiation_info': {'secure_renegotitation': ''},
    '_selector': 0xff01},
   {'ext_supported_points': {'supported_points': 0x00}, '_selector': 0x000b},
   {'ext_session_tickets': '', '_selector': 0x0023},
   {'use_srtp': {'data': [0x0001], 'mki': ''}, '_selector': 0x000e}]},
 '_selector': 0x02}

In [25]:
# rem = bytes.fromhex('16fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda003020102020900b388ba8f0397e380300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303931365a170d3234303731353039303931365a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d030107034200045d4f0a2e4e3e0bfe83012c62d4649a43fa9bb46aa4579fa59b739e6463a0a8c0395545195f81f8895480128cda85a9c79699baca8b413065c7ef013c226be5e6300a06082a8648ce3d040302034800304502206e8136052a4d8689988d3f1205882d1d224cd9b2543c1e9463ac9bae5aee87f90221009c4d8b5e119541c461c4d704121abb097ee983dddb0c7ea4c8fd06ba08623cea')
packet, rem1 = public_match(dtls_record_spec, rem)
transcript += rem[13:-len(rem1)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000001,
 'tls_handshake': {'type_and_length': 0x0b00011f,
  'message_sequence': 0x0001,
  'fragment_offset': 0x000000,
  'fragment': 0x00011c000119308201153081bca003020102020819cb06460f58e117300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303632353132323631355a170d3234303732363132323631355a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004a121b769592183e9b708bee419d6b5dc5eea855a34b7df34b87aa9afdd00870f2264e0686761facd7f360ff3269083fccb235c9ca5f1609c6b3c94478b19a196300a06082a8648ce3d04030203480030450220652d2414f9a0fd5f59964b954270a89b2058cdc9d39406006eec2fae1986de640221008230cebb0a69bf102ca89d89c3994b60341d91141c0d47543954d2433640e385}}

In [26]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
peer_cert, _ = public_match(handshake_spec, record)
peer_cert

{'certificates_data': {'certificates': [{'certificate': 0x308201153081bca003020102020819cb06460f58e117300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303632353132323631355a170d3234303732363132323631355a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004a121b769592183e9b708bee419d6b5dc5eea855a34b7df34b87aa9afdd00870f2264e0686761facd7f360ff3269083fccb235c9ca5f1609c6b3c94478b19a196300a06082a8648ce3d04030203480030450220652d2414f9a0fd5f59964b954270a89b2058cdc9d39406006eec2fae1986de640221008230cebb0a69bf102ca89d89c3994b60341d91141c0d47543954d2433640e385}]},
 '_selector': "
 "}

In [27]:

first_server_cert = None
for cert in peer_cert['certificates_data']['certificates']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].bytes())
    if first_server_cert is None:
        first_server_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 1858586169195421975 (0x19cb06460f58e117)
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jun 25 12:26:15 2024 GMT
            Not After : Jul 26 12:26:15 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:a1:21:b7:69:59:21:83:e9:b7:08:be:e4:19:d6:
                    b5:dc:5e:ea:85:5a:34:b7:df:34:b8:7a:a9:af:dd:
                    00:87:0f:22:64:e0:68:67:61:fa:cd:7f:36:0f:f3:
                    26:90:83:fc:cb:23:5c:9c:a5:f1:60:9c:6b:3c:94:
                    47:8b:19:a1:96
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:45:02:20:65:2d:24:14:f9:a0:fd:5f:59:96:4b:95:42:70:
        a8:9b:20:58:cd:c9:d3:94:06:

In [28]:
first_server_cert.digest('sha256')
# TODO: assert digest

b'E0:D8:A2:3B:54:91:BF:55:42:CC:C0:D5:73:7E:98:CA:4E:83:64:3A:EF:42:6C:5A:2F:A8:66:4E:39:52:20:18'

In [29]:
packet, rem2 = public_match(dtls_record_spec, rem1)
transcript += rem1[13:-len(rem2)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000002,
 'tls_handshake': {'type_and_length': 0x0c000070,
  'message_sequence': 0x0002,
  'fragment_offset': 0x000000,
  'fragment': 0x03001d20cb0aefea1b9a86220ab2d2ca76454aef224da8e0369ed84ea92ad8bc3a0d3c720403004830460221008f333de48adbcd9ba006bea0ddff31801efe39383f2e521ee68700ff2347eeda022100e95735270ab121ba2be5ed9613137309c253af48a02cfc7c332b749712c19b45}}

In [30]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_key, _ = public_match(handshake_spec, record)
server_key

{'server_key_exchange': {'curve_type': 0x03,
  'named_curve': 0x001d,
  'pubkey': 0xcb0aefea1b9a86220ab2d2ca76454aef224da8e0369ed84ea92ad8bc3a0d3c72,
  'sig_algorithm': 0x0403,
  'signature': 0x30460221008f333de48adbcd9ba006bea0ddff31801efe39383f2e521ee68700ff2347eeda022100e95735270ab121ba2be5ed9613137309c253af48a02cfc7c332b749712c19b45},
 '_selector': "
 "}

In [31]:
server_key_sign_spec = """32B@client_random-32B@server_random-8@curve_type-16@named_curve-8@pubkey()"""
server_key_sign = {
    "client_random": client_hello['client_hello']['random'],
    "server_random": server_hello['server_hello']['random'],
    "curve_type": server_key['server_key_exchange']['curve_type'],
    "named_curve": server_key['server_key_exchange']['named_curve'],
    "pubkey": server_key['server_key_exchange']['pubkey'],
}

enc = public_construct(server_key_sign_spec, server_key_sign)

first_server_cert.get_pubkey().to_cryptography_key().verify(server_key['server_key_exchange']['signature'].bytes(), enc, ec.ECDSA(hashes.SHA256()))


In [32]:
packet, rem3 = public_match(dtls_record_spec, rem2)
transcript += rem2[13:-len(rem3)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000003,
 'tls_handshake': {'type_and_length': 0x0d000019,
  'message_sequence': 0x0003,
  'fragment_offset': 0x000000,
  'fragment': 0x02014000120403080404010503080505010806060102010000}}

In [33]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
certificate_request, _ = public_match(handshake_spec, record)
certificate_request

{'certificate_request': {'cert_types': [{'cert_type': 0x01},
   {'cert_type': "@"}],
  'sig_hash_algs': [{'sig_hash_alg': 0x0403},
   {'sig_hash_alg': 0x0804},
   {'sig_hash_alg': 0x0401},
   {'sig_hash_alg': 0x0503},
   {'sig_hash_alg': 0x0805},
   {'sig_hash_alg': 0x0501},
   {'sig_hash_alg': 0x0806},
   {'sig_hash_alg': 0x0601},
   {'sig_hash_alg': 0x0201}],
  'distinguished_names': ''},
 '_selector': "
 "}

In [34]:
packet, rem4 = public_match(dtls_record_spec, rem3)
transcript += rem3[13:]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000004,
 'tls_handshake': {'type_and_length': 0x0e000000,
  'message_sequence': 0x0004,
  'fragment_offset': 0x000000,
  'fragment': ''}}

In [35]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_hello_done, _ = public_match(handshake_spec, record)
server_hello_done

{'server_hello_done': '', '_selector': 0x0e}

In [36]:
dtls_message_encoded = b''

In [37]:
certificate = {
    'certificates_data':
        {
            'certificates':
                 [{'certificate': OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_ASN1, local_cert)}]
        },
 '_selector': 11
}

certificate_encoded = public_construct(handshake_spec, certificate)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_encoded[:4],
        'message_sequence': 1,
        'fragment_offset': 0,
        'fragment': certificate_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 1,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [38]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey

client_private_key = X25519PrivateKey.generate()
client_public_key = client_private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)


In [39]:

client_key_exchange = {
    'client_key_exchange': {
        'pubkey': client_public_key,
    },
 '_selector': 16
}

client_key_exchange_encoded = public_construct(handshake_spec, client_key_exchange)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_key_exchange_encoded[:4],
        'message_sequence': 2,
        'fragment_offset': 0,
        'fragment': client_key_exchange_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 2,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [40]:
signature = own_cert_private_key.sign(transcript, ec.ECDSA(hashes.SHA256()))

In [41]:

certificate_verify = {
    'certificate_verify': {
        'sig_algorithm': 0x0403,
        'signature': signature,
    },
 '_selector': 15
}

certificate_verify_encoded = public_construct(handshake_spec, certificate_verify)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_verify_encoded[:4],
        'message_sequence': 3,
        'fragment_offset': 0,
        'fragment': certificate_verify_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 3,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
# transcript += dtls_record_encoded[13:]
certificate_verify_message_encoded = dtls_record_encoded
dtls_message_encoded += dtls_record_encoded

In [42]:

dtls_record = {
    "tls_version": 0xfefd,
    "epoch": 0,
    "sequence_number": 4,
    "change_cipher_spec_message": 0x01,
    "_selector": 20
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
dtls_message_encoded += dtls_record_encoded

In [43]:
from cryptography.hazmat.primitives import hashes, hmac

def HMAC_SHA256(secret, val):
    h = hmac.HMAC(secret, hashes.SHA256())
    h.update(val)
    return h.finalize()

def P_SHA256(secret, seed, l):
    b = bytearray()
    A = HMAC_SHA256(secret, seed)
    while len(b) < l:
        b += HMAC_SHA256(secret, A + seed)
        A = HMAC_SHA256(secret, A)
    return b[:l]

def PRF(secret, label, seed, l):
    return P_SHA256(secret, label + seed, l)

In [44]:
def calculate_sha256(data):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(data)
    return digest.finalize()

In [45]:
shared_key = client_private_key.exchange(X25519PublicKey.from_public_bytes(server_key['server_key_exchange']['pubkey'].bytes()))
print("calculated shared key", shared_key)

calculated shared key b'(\xb5\xef\x10\xe3\xd5\x14\xaf%yaL\x07v\xd0\x95\r%.b\xe7\xc1\xf2\x98\xb0\x0e\x8fN~#\xab&'


In [46]:
master_key = PRF(
    shared_key,
    b"extended master secret",
    # client_hello['client_hello']['random'].bytes() + server_hello['server_hello']['random'],
    calculate_sha256(transcript),
    48,
)
master_key.hex()

'2d749ef4fc00065b644767f40ff44e2727537d8ac6ad5a480d24d2195157644cbd8acb5456a0f8d0aa41c1d923eeace6'

In [47]:
keyLen = 16
ivLen  = 4

raw_keys = PRF(
    master_key,
    b"key expansion",
    server_hello['server_hello']['random'].bytes() + client_hello['client_hello']['random'],
    40,
)

client_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

server_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

client_write_iv = raw_keys[:ivLen]
raw_keys = raw_keys[ivLen:]

server_write_iv = raw_keys[:ivLen]

In [48]:
transcript += certificate_verify_message_encoded[13:]

In [49]:
client_verify_data = PRF(master_key, b"client finished", calculate_sha256(transcript), 12)
client_verify_data.hex()

'51ad700c49510432e160475a'

In [50]:
encrypted_dtls_record_spec = """
8@content_type-16@tls_version-16@epoch-6B@sequence_number-16@encrypted_data()
"""

In [51]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

aad_spec = """16@epoch-6B@sequence_number-8@content_type-16@tls_version-16@data_length"""

def decrypt_packet(packet, key, iv):
    encrypted = packet['encrypted_data']
    if not isinstance(encrypted, bytes):
        encrypted = encrypted.bytes()

    aad = packet.copy()
    aad['data_length'] = len(encrypted) - 16 - 8
    aad_encoded = bytearray(public_construct(aad_spec, aad))
    aad_encoded[:8] = encrypted[:8]

    cipher = AESGCM(key)
    all_server_cert_packet_encoded = cipher.decrypt(iv + encrypted[:8], encrypted[8:], aad_encoded)
    if packet['content_type'].int() == 0x16:
        all_server_cert_packet, _ = public_match(fragment_spec, all_server_cert_packet_encoded)
        packet['tls_handshake'] = all_server_cert_packet
    else:
        packet['decrypted'] = all_server_cert_packet_encoded
    return packet
    

def encrypt_packet(packet, key, iv):
    if packet['content_type'] == 0x16:
        to_encrypt = public_construct(fragment_spec, packet['tls_handshake'])
    else:
        to_encrypt = packet['decrypted']

    aad = packet.copy()
    aad['data_length'] = len(to_encrypt)
    # aad['sequence_number'] = aad['tls_handshake']['message_sequence']
    aad_encoded = public_construct(aad_spec, aad)
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(iv + aad_encoded[:8], to_encrypt, aad_encoded)
    packet['encrypted_data'] = aad_encoded[:8] + encrypted_app_data
    packet['data_length'] = len(packet['encrypted_data'])
    return packet

In [52]:
finshed_record = {
    "tls_version": 0xfefd,
    "epoch": 1,
    "sequence_number": 0,
    "tls_handshake": {
        'type_and_length': 0x1400000c,
      'message_sequence': 4,
      'fragment_offset': 0x000000,
      'fragment': bytes(client_verify_data)
    },
    "content_type": 22,
    "_selector": 22,
}
encrypted_finished_record = encrypt_packet(finshed_record, client_write_key, client_write_iv)
print(encrypted_finished_record)
encrypted_dtls_record_encoded = public_construct(encrypted_dtls_record_spec, encrypted_finished_record)

# transcript += dtls_record_encoded[13:]
dtls_message_encoded += encrypted_dtls_record_encoded

{'tls_version': 65277, 'epoch': 1, 'sequence_number': 0, 'tls_handshake': {'type_and_length': 335544332, 'message_sequence': 4, 'fragment_offset': 0, 'fragment': b'Q\xadp\x0cIQ\x042\xe1`GZ'}, 'content_type': 22, '_selector': 22, 'encrypted_data': b'\x00\x01\x00\x00\x00\x00\x00\x00tz\xd4\x03\xcd\x15\xd7\x97\xa9\xfd\x8cDe\xcf=\x13\xa8E#\x97\x0b\x18\xe6n$\xd3\xe0\xfd\xc8do}}gv\xd0%]\xb5\xf5', 'data_length': 48}


In [53]:
aaa = public_construct(dtls_record_spec, finshed_record)
print(aaa[13:])
transcript += aaa[13:]
# transcript = transcript[:-len(aaa)+13]

b'\x14\x00\x00\x0c\x00\x04\x00\x00\x00\x00\x00\x0cQ\xadp\x0cIQ\x042\xe1`GZ'


In [54]:
sock.send(dtls_message_encoded)
dtls_message_encoded = b''

In [58]:
msg = dts_queue.get()
msg

b'\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00\n\x01\xe2\x04\x00\x01\xd6\x00\x05\x00\x00\x00\x00\x01\xd6\x00\x00\x1c\x1f\x01\xd0T\xf1\x9a\xb5m\xd3r\xf67\xf8\x15\x9ao:\x1a\xd9\xe0\x1f{\xdf\xed%\xb5}5VJ\x93\xef\xeb\x0e\x99\xe1\xa19\x84\x84(\xa94\x9c\xdfq\x06\x9a)\xe2\xd7\xf2\x13b\xd0\x9a\xd6\xbayG\xea\x07"\xee\xa3\xe68m\xd4\x0e\xe2#\xc4k.\xf0c\xb5\x1e\xa8\xbf\xe7\xbd\x05X\xd8\xd8\x1be\xa0\xfeH>\x02\xf0\xfd.\xea\x8e"\x84!\x84\xb2\x8e\xa6x\x12\xcfO\x1f3\xc9D\x8c\x9d\xf8m\x85\x0e\xf1\xe3\x8e\x06\x80\x94\x83\xfc>\xe0\x0f\xd8\x87]\xc3\xeb9\x02Ca\x8e\xbe\xfd\r=\xd9@:\xae\x1ci?\xa5 ,\xc5\x9c\x7f\xce\xc6\xc3cx\xcb\xeb\x9d|Q\xda\xc8\x81k\xd6\x84\xf5l\xee2\x99\r\x86t\xf4\x94\xc5\xf3H\xd26l\xb7\x81\xc1AgiC\xbc\xfcq\xeb\xd6#6z\xf2\xc2\x0f>\x16\x00N\xcb\xff\xc0Q\xf4\xdda`^;\x07C\xd7\xf1\xd068\xac\x98\x9c5cZ\xf2"H\xc5\x02E\x9a\x81\xd4v\x1d\xb9\xa5\x19x\xdd\xa0>\x08w\xdcS\x03^\x9a\xd7\xaag\x1e\xefB\x85>Y\xf5\xc3\xbf\x081>\xc71(!\xbbPo\x90\x99i\xfb\xfd\xa5\xc6\xbeV@\xf6\xc4\x08\x87{\x1d\x02\xf7j_\xe8\xce\n\

In [59]:
packet, rem = public_match(dtls_record_spec, msg)
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
peer_cert, _ = public_match(handshake_spec, record)
peer_cert

{'new_session_ticket': {'lifetime_hint': 0x00001c1f,
  'session_ticket': 0x54f19ab56dd372f637f8159a6f3a1ad9e01f7bdfed25b57d35564a93efeb0e99e1a139848428a9349cdf71069a29e2d7f21362d09ad6ba7947ea0722eea3e6386dd40ee223c46b2ef063b51ea8bfe7bd0558d8d81b65a0fe483e02f0fd2eea8e22842184b28ea67812cf4f1f33c9448c9df86d850ef1e38e06809483fc3ee00fd8875dc3eb390243618ebefd0d3dd9403aae1c693fa5202cc59c7fcec6c36378cbeb9d7c51dac8816bd684f56cee32990d8674f494c5f348d2366cb781c141676943bcfc71ebd623367af2c20f3e16004ecbffc051f4dd61605e3b0743d7f1d03638ac989c35635af22248c502459a81d4761db9a51978dda03e0877dc53035e9ad7aa671eef42853e59f5c3bf08313ec7312821bb506f909969fbfda5c6be5640f6c408877b1d02f76a5fe8ce0a1a34c7ba7616e7c27457e121c97db12256f4314b4f325ad931e5641d12738b2529dce1b8dd0d575d91895154715de29aeb65c258e69d0d13a63e2c26ca565e355a22d0eb5c71dc42167e5d760a0171af6d7f6a0d8910018543eb4262df22c4d831ed5224af3e01965d118646607579b81bb26d14103992af51f09fd098e018e5f149f5862ff781ff6088395b5d42e25e6ad9dbc92c2d0e98176cd92a5100732a5

In [60]:
if 'new_session_ticket' not in peer_cert:
    raise RuntimeError("repeated package, retry from dts_queue.get()")
transcript += msg[13:-len(rem)]


In [61]:
packet, rem1 = public_match(dtls_record_spec, rem)
packet

{'tls_version': 0xfefd,
 '_selector': 0x14,
 'epoch': 0x0000,
 'sequence_number': 0x00000000000b,
 'change_cipher_spec_message': 0x01}

In [62]:
packet, rem2 = public_match(encrypted_dtls_record_spec, rem1)
packet

{'content_type': 0x16,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x0001000000000000f204a9ac6ac37fcb71f32ba0a0dfb7702b10536152b77427c1d7277af624812a0eb19244318e2965}

In [63]:
finished_fragment = decrypt_packet(
    packet,
    server_write_key,
    server_write_iv,
)
finished_fragment

{'content_type': 0x16,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x0001000000000000f204a9ac6ac37fcb71f32ba0a0dfb7702b10536152b77427c1d7277af624812a0eb19244318e2965,
 'tls_handshake': {'type_and_length': 0x1400000c,
  'message_sequence': 0x0006,
  'fragment_offset': 0x000000,
  'fragment': 0x0e42f84ca0f5f6cb66af7d66}}

In [64]:
server_verify_data = PRF(master_key, b"server finished", calculate_sha256(transcript), 12)
server_verify_data.hex()

'0e42f84ca0f5f6cb66af7d66'

In [65]:
assert finished_fragment['tls_handshake']['fragment'].bytes() == server_verify_data, "server verify data mismatch"

In [66]:
import crc

In [67]:
sctp_chunks_spec = """
8==1?8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
:8==2?8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
:8==10?8@chunk_flags
    -16@chunk_length
    -@cookie
:8==11?8@chunk_flags
    -16@chunk_length
:8==0?4@res
    -1@i-1@u-1@b-1@e
    -16@chunk_length
    -32@tsn
    -16@s-16@n
    -32@payload_protocol_id
    -@data
:8==3?8@flags
    -16@chunk_length
    -32@tsn_ack
    -32@a_rwnd
    -16@n_gap_ack_blocks
    -16@n_duplicate_tsns
    -@gap_ack_duplicate_tsns
:8==4?8@flags
    -16@chunk_length
    -@optional_attributes_encoded
:8==5?8@flags
    -16@chunk_length
    -@optional_attributes_encoded
"""

optional_header_spec = """16@type-16@length"""
sctp_chunk_header_spec = "8@chunk_type-8@chunk_flags-16@chunk_size"

def parse_sctp_chunk(sctp_chunk):
    sctp_chunk_header, _ = public_match(sctp_chunk_header_spec, sctp_chunk)
    data_len = sctp_chunk_header['chunk_size'].int()
    sctp_chunk_value, sctp_chunk = sctp_chunk[:data_len], sctp_chunk[math.ceil(data_len /4) * 4:]
    res, _ = public_match(sctp_chunks_spec, sctp_chunk_value)
    if 'optional_attributes_encoded' in res:
        oares = res['optional_attributes_encoded'].bytes()
        res['optional_attributes'] = []
        while oares:
            optional_attribute_header, _ = public_match(optional_header_spec, oares)
            optional_attribute_header['data'] = oares[4:optional_attribute_header['length'].int()]
            oares = oares[optional_attribute_header['length'].int():]
            res['optional_attributes'].append(optional_attribute_header)
    return res, sctp_chunk
            
        

In [68]:
sctp_common_header_spec = """
16@source_port
-16@dst_port
-32@verification_tag
-32@checksum"""

def get_sctp_message(dtls_message):
    packet, _ = public_match(encrypted_dtls_record_spec, dtls_message)
    decrypted = decrypt_packet(
        packet,
        server_write_key,
        server_write_iv,
    )
    raw = decrypted['decrypted']
    raw_clone = bytearray(raw)
    raw_clone[8:12] = b'\0\0\0\0'
    crc_res = public_construct('32@data', {'data': crc.crc(raw_clone)})[::-1]
    
    sctp_header, rem = public_match(sctp_common_header_spec, raw)
    assert crc_res == sctp_header['checksum'].bytes(), "invalid checksum"
    chunks = []
    while rem:
        chunk, rem = parse_sctp_chunk(rem)
        chunks.append(chunk)
    return {"header": sctp_header, "chunks": chunks, "epoch": decrypted["epoch"], "sequence_number": decrypted["sequence_number"]}

In [69]:
optional_full_header_spec = """16@type-16@length-@data"""
def construct_sctp_chunk(sctp_chunk):
    if 'optional_attributes' in sctp_chunk:
        sctp_chunk['optional_attributes_encoded'] = b''
        for attr in sctp_chunk['optional_attributes']:
            attr['length'] = len(attr['data']) + 4
            sctp_chunk['optional_attributes_encoded'] += public_construct(optional_full_header_spec, attr)
    sctp_chunk['chunk_length'] = 0
    res = public_construct(sctp_chunks_spec, sctp_chunk)
    sctp_chunk['chunk_length'] = len(res)
    res = public_construct(sctp_chunks_spec, sctp_chunk)
    if sctp_chunk['chunk_length'] % 4 != 0:
        res += b'\0' * (4-(sctp_chunk['chunk_length']%4))
    return res
    

In [70]:
def create_sctp_message(sctp_message):
    sctp_message['header']['checksum'] = 0
    buff = public_construct(sctp_common_header_spec, sctp_message['header'])
    for chunk in sctp_message['chunks']:
        buff += construct_sctp_chunk(chunk)

    buff = bytearray(buff)
    crc_res = public_construct('32@data', {'data': crc.crc(buff)})[::-1]
    buff[8:12] = crc_res
    
    packet = {
        "tls_version": 0xfefd,
        "epoch": sctp_message["epoch"],
        "sequence_number": sctp_message["sequence_number"],
        "decrypted": bytes(buff),
        "content_type": 23,
        "_selector": 23,
    }
    encrypted = encrypt_packet(
        packet,
        client_write_key,
        client_write_iv,
    )
    return public_construct(encrypted_dtls_record_spec, encrypted)
    

In [71]:
while True:
    msg = dts_queue.get()
    if msg[0] != 0x17:
        print('not an app data, retrying')
        continue
    print('got app data')
    break

sctp_init = get_sctp_message(msg)
client_verification_tag = sctp_init['chunks'][0]['initiate_tag']
sctp_init

got app data


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0x00000000,
  'checksum': 0x89414bc9},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 0x01,
   'chunk_length': 0x001e,
   'initiate_tag': 0x9fae03e2,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': 0x10c7e9d0,
   'optional_attributes_encoded': 0xc00000048008000682c0,
   'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
    {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000001}

In [72]:
tsn = 0xe306eef2
own_dtls_seq = 2

In [73]:
# sctp_init = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': 0x00000000,
#   'checksum': 0x9e2630c2},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 0x01,
#    'chunk_length': 0x001e,
#    'initiate_tag': 0x6ee08c30,
#    'a_rwnc': 0x00500000,
#    'num_out_streams': 0xffff,
#    'num_in_streams': 0xffff,
#    'initial_tsn': tsn,
#    'optional_attributes_encoded': 0xc00000048008000682c0,
#    'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
#     {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000007})
# sock.send(sctp_init)

In [74]:
sctp_init_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x378def6c},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 0x02,
   'chunk_length': 0x0052,
   'initiate_tag': 0xff448058,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': tsn,
   'optional_attributes_encoded': 0x0007003164635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00000000c00000048008000682c0,
   'optional_attributes': [{'type': 0x0007,
     'length': 0x0031,
     'data': b'dcSCTP00n\xe0\x8c0\xffD\x80X\x91p\x92\x7f\xe3\x06\xee\xf2\x00P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\xff\xff\xff\xff\x00'},
    {'type': 0x0000,
     'length': 0x00c0,
     'data': b'\x00\x00\x04\x80\x08\x00\x06\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_init_ack)


In [75]:
# while True:
#     msg = dts_queue.get()
#     pkt = get_sctp_message(msg)
#     if pkt['chunks'][0]['_selector'].int() != 2:
#         continue
#     print('found init ack')    
#     cookie = next(i['data'] for i in pkt['chunks'][0]['optional_attributes'] if i['type'].int() == 7)
#     server_verification_tag = pkt['header']['verification_tag']
#     break
# pkt

In [76]:
while True:
    msg = dts_queue.get()
    pkt = get_sctp_message(msg)
    if pkt['chunks'][0]['_selector'].int() != 10:
        print('not found cookie echo, got', pkt['chunks'][0]['_selector'].int(), "instead")    
        continue
    print('found cookie echo')
    break
pkt

not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
found cookie echo


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xa804049c},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': "
   ",
   'chunk_length': 0x0031,
   'cookie': 0x64635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000005}

In [77]:
# sctp_cookie_echo = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': server_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 10,
#     'cookie': cookie}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_echo)

In [78]:
sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 11}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_cookie_ack)

In [79]:
# sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': client_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 11}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_ack)

In [80]:
sctp_data_sequence_number = 0

In [81]:
data_send = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x14fad29b},
 'chunks': [{'res': 0b0000,
   '_selector': 0x00,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': tsn,
   's': 0x0000,
   'n': sctp_data_sequence_number,
   'payload_protocol_id': 0x00000033,
   'data': "hello, my name is mie2"}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
tsn += 1
sctp_data_sequence_number += 1
sock.send(data_send)

In [82]:
msg = dts_queue.get()
data_ack_received = get_sctp_message(msg)
data_ack_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xe02faf3e},
 'chunks': [{'flags': 0x00,
   '_selector': 0x03,
   'chunk_length': 0x0010,
   'tsn_ack': 0xe306eef2,
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': ''}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000006}

In [83]:
msg = dts_queue.get()
data_received = get_sctp_message(msg)
data_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': "Cb$G"},
 'chunks': [{'res': 0b0000,
   '_selector': 0x00,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': 0x10c7e9d0,
   's': 0x0000,
   'n': 0x0000,
   'payload_protocol_id': 0x00000033,
   'data': "asd"}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000007}

In [84]:
sack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0xe02faf3e},
 'chunks': [{'flags': 0x00,
   '_selector': 0x03,
   'chunk_length': 0x0010,
   'tsn_ack': data_received['chunks'][-1]['tsn'],
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': ''}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sack)

In [85]:
msg = dts_queue.get()
heartbeat_received = get_sctp_message(msg)
heartbeat_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0x90f0a6fc},
 'chunks': [{'flags': 0x00,
   '_selector': 0x04,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c00000000ca792f4a,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': b'\x00\x00\x00\x00\xcay/J'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000008}

In [86]:
heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x30743be0},
 'chunks': [{'flags': 0x00,
   '_selector': 0x05,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c00000000a92c0797,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': heartbeat_received['chunks'][0]['optional_attributes'][0]['data']}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(heartbeat_ack)

In [87]:
background_sctp_handler_running = False
try:
    background_sctp_thread.join()
except:
    pass
def message_handler(msg):
    global own_dtls_seq
    decoded = get_sctp_message(msg)
    ret = []
    for chunk in decoded['chunks']:
        if chunk['_selector'].int() == 0: # data
            sack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0xe02faf3e},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x03,
                    'chunk_length': 0x0010,
                    'tsn_ack': chunk['tsn'],
                    'a_rwnd': 0x00480000,
                    'n_gap_ack_blocks': 0x0000,
                    'n_duplicate_tsns': 0x0000,
                    'gap_ack_duplicate_tsns': ''}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(sack)
            ret.append(chunk['data'])
        elif chunk['_selector'].int() == 4: # heartbeat
            heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0x30743be0},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x05,
                    'chunk_length': 0x0010,
                    'optional_attributes_encoded': 0x0001000c00000000a92c0797,
                    'optional_attributes': [{'type': 0x0001,
                    'length': 0x000c,
                    'data': chunk['optional_attributes'][0]['data']}]}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(heartbeat_ack)
    return ret


In [88]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        print(msg)
        print(message_handler(msg))
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()

b"\x17\xfe\xfd\x00\x01\x00\x00\x00\x00\x00\t\x004\x00\x01\x00\x00\x00\x00\x00\t\xa0\x95\xae\xed\xfe\xd6\xbb'0L\x03^\xcbK\xcbD\xb0\xe7\x00\x08\x03\xc3s\t\xee]\xb1`\xd8T\x89:\xd3\x0b\xb1\xedtY7\x18\xdf.P\xcb"
[]
exiting


In [89]:
def send_message(data):
    global own_dtls_seq
    global tsn
    global sctp_data_sequence_number
    data_send = create_sctp_message({'header': {'source_port': 0x1388,
        'dst_port': 0x1388,
        'verification_tag': client_verification_tag,
        'checksum': 0x14fad29b},
        'chunks': [{'res': 0b0000,
            '_selector': 0x00,
            'i': 0b0,
            'u': 0b0,
            'b': 0b1,
            'e': 0b1,
            'chunk_length': 0x0013,
            'tsn': tsn,
            's': 0x0000,
            'n': sctp_data_sequence_number,
            'payload_protocol_id': 0x00000033,
            'data': data}],
        'epoch': 0x0001,
        'sequence_number': own_dtls_seq})
    own_dtls_seq += 1
    tsn += 1
    sctp_data_sequence_number += 1
    sock.send(data_send)

In [90]:
send_message("vzxcasd")

In [91]:
background_sctp_handler_running = False
background_sctp_thread.join()

In [92]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        ret = message_handler(msg)
        for i in ret:
            print('>', i.str())
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()
while True:
    send_message(input())

 hi


> hello


KeyboardInterrupt: Interrupted by user

Exception in thread Thread-8 (background_sctp_handler):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6512/311264759.py", line 5, in background_sctp_handler
  File "/tmp/ipykernel_6512/3601383506.py", line 8, in message_handler
  File "/tmp/ipykernel_6512/2449173507.py", line 23, in get_sctp_message
  File "/tmp/ipykernel_6512/2343539820.py", line 50, in parse_sctp_chunk
  File "/home/mie/notebooks/parser.py", line 368, in public_match
    obj, rem = all_match(process_spec(spec), bytes2bin(data), allow_rem=True)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mie/notebooks/parser.py", line 